<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250"/>

# TB - DevOps: Mise en place de cloud-native storage
## fio benchmark
La version de fio utilisée pour chaque benchmark va être explicitement décrite tel que la license morale (section 4 [documentation](https://fio.readthedocs.io/en/latest/fio_doc.html#moral-license) fio) nous recommande.

## Local
Les résultats de cette section ont été généré à l'aide de fio version 3.16 à l'aide du script `run-all-jobs.sh`.

In [1]:
import json

job_names = ['randomread', 'randomwrite', 'sequentialread']
job_outputs_filename = [f"{job}.fio-output" for job in job_names]

In [2]:
job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"fio-jobs/{job_output}")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])

{'fio version': 'fio-3.16', 'timestamp': 1624715980, 'timestamp_ms': 1624715980268, 'time': 'Sat Jun 26 15:59:40 2021', 'global options': {'bs': '4K', 'iodepth': '256', 'direct': '1', 'ioengine': 'libaio', 'runtime': '120', 'numjobs': '4', 'size': '512M', 'name': 'raw-readread', 'rw': 'randread'}, 'jobs': [{'jobname': 'job1', 'groupid': 0, 'error': 0, 'eta': 0, 'elapsed': 121, 'job options': {'filename': '/dev/nvme0n1p7'}, 'read': {'io_bytes': 199040774144, 'io_kbytes': 194375756, 'bw_bytes': 1658617830, 'bw': 1619743, 'iops': 404935.9938, 'runtime': 120004, 'total_ios': 48593939, 'short_ios': 0, 'drop_ios': 0, 'slat_ns': {'min': 728, 'max': 3734933, 'mean': 1513.3561, 'stddev': 1458.146573}, 'clat_ns': {'min': 51367, 'max': 12383842, 'mean': 2526780.637604, 'stddev': 589803.058981, 'percentile': {'1.000000': 733184, '5.000000': 2056192, '10.000000': 2146304, '20.000000': 2211840, '30.000000': 2277376, '40.000000': 2342912, '50.000000': 2408448, '60.000000': 2473984, '70.000000': 26050

## Cluster IICT
TODO option directory=???

On souhaite déployer un container docker avec un PV dans le cluster IICT. Le container contient l'outil *fio* et un dossier *fio-jobs*.

Ayant signé la clause de confidentialité, on ne vas pas publier le repo git

`# docker build --tag fiobench . && docker run -d fiobench`

Architecting it fait des [scripts](https://github.com/architectingit/k8sstorage/blob/main/perfraw.sh) et pas de fio-jobs. C'est vrai que ça peut être plus "rapide" mais ca ne rend pas facilement testable.
### Connection au cluster IICT
Connectez-vous au VPN de l'école, puis:
```bash
kubectl config get-contexts # is ` iict ` listed ?
kubectl config use-context iict
```

On tente de run une image simple:

```bash
kubectl apply -f iict-fio-benchmark-manual.yaml
```

### Manual
Observons quel classe de stockage sont disponibles:
```bash
$ kubectl get storageclass
NAME                 PROVISIONER                                     RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
longhorn (default)   driver.longhorn.io                              Delete          Immediate           true                   170d
nfs-client           cluster.local/nfs-subdir-external-provisioner   Delete          Immediate           true                   114d
```

Nous allons donc faire un PVC de storageclass `longhorn`.

Ensuite, on déploie un container simple:

```bash
$ kubectl apply --namespace=mercado -f iict-fio-benchmark-manual.yaml
```

On vérifie que le container a bien démarré:
```bash
$ kubectl get pods --namespace=mercado
```

On ouvre une session interactive et on install fio:
```bash
$ kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
/ # apk add fio
/ # fio --version
$ exit # on sort de la session
```

On va copier un fio-job:
```bash
$ kubectl --namespace=mercado cp docker/iict/run-all-jobs.sh $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/

# ou 6 lignes:
kubectl --namespace=mercado cp docker/iict/fio-jobs/random-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
kubectl --namespace=mercado cp docker/iict/fio-jobs/random-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
kubectl --namespace=mercado cp docker/iict/fio-jobs/random-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-read.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-readwrite.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
kubectl --namespace=mercado cp docker/iict/fio-jobs/sequential-write.fio $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/
```

Et on exécute les tests:
```bash
# ouvrir une session interactive
kubectl exec --namespace=mercado -it $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}') -- /bin/sh
sh run-all-jobs.sh
exit
```

On récupère tous les outputs:
```bash
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-read.fio-output fio-jobs/iict/random-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-readwrite.fio-output fio-jobs/iict/random-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/random-write.fio-output fio-jobs/iict/random-write.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-read.fio-output fio-jobs/iict/sequential-read.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-readwrite.fio-output fio-jobs/iict/sequential-readwrite.fio-output
kubectl --namespace=mercado cp $(kubectl get pods --namespace=mercado -o=jsonpath='{.items[0].metadata.name}'):/sequential-write.fio-output fio-jobs/iict/sequential-write.fio-output
```

Une fois que l'on a terminé, on nettoie:
```bash
kubectl delete -f iict-fio-benchmark-manual.yaml
```

Maintenant on va visualiser nos données:


In [2]:
job_outputs_filename = ['random-read.fio-output', 'random-readwrite.fio-output', 'random-write.fio-output', 'sequential-read.fio-output', 'sequential-readwrite.fio-output', 'sequential-write.fio-output']

In [4]:
import json

job_outputs = []

# Load the results
for job_output in job_outputs_filename:
    f = open(f"fio-jobs/iict/{job_output}")
    data = json.load(f)
    job_outputs.append(data)

print(job_outputs[0])


{'fio version': 'fio-3.27', 'timestamp': 1624891230, 'timestamp_ms': 1624891230621, 'time': 'Mon Jun 28 14:40:30 2021', 'global options': {'bs': '4K', 'iodepth': '256', 'direct': '1', 'ioengine': 'libaio', 'runtime': '120', 'numjobs': '4', 'size': '512M', 'name': 'random-read', 'rw': 'randread', 'directory': '/data'}, 'jobs': [{'jobname': 'job1', 'groupid': 0, 'error': 0, 'eta': 0, 'elapsed': 121, 'read': {'io_bytes': 6757625856, 'io_kbytes': 6599244, 'bw_bytes': 56305572, 'bw': 54985, 'iops': 13746.477582, 'runtime': 120017, 'total_ios': 1649811, 'short_ios': 0, 'drop_ios': 0, 'slat_ns': {'min': 1387, 'max': 680893494, 'mean': 288339.033804, 'stddev': 2637859.661876, 'N': 1649811}, 'clat_ns': {'min': 4805572, 'max': 3976799704, 'mean': 74142898.105001, 'stddev': 81724907.285792, 'N': 1649811, 'percentile': {'1.000000': 45875200, '5.000000': 53215232, '10.000000': 56885248, '20.000000': 61603840, '30.000000': 64749568, '40.000000': 67633152, '50.000000': 69730304, '60.000000': 72876032

TODO copier les jobs de [architectingit](https://github.com/architectingit/k8sstorage/blob/main/perfraw.sh) et comparer I guess.

Note: [google](https://cloud.google.com/compute/docs/disks/benchmarking-pd-performance) persistent disk benchmark